In [1]:
import pandas as pd
import numpy as np
import time
import h5py
from sklearn.feature_selection import VarianceThreshold

In [2]:
#Load full raw data
full_d = pd.read_pickle('../DATA/tumor_data.pkl')

#If we are going to do the sorting, here might be a good place to do it


#All genes to csv
pd.DataFrame(full_d.index.values).to_csv("genes.csv", index=False, header=False)

#Log2 transform and variance thresholding
var = np.var(np.log2(full_d+1), axis=1)
ind2drop = var.sort_values().index[0:-10404] #sorts from smalles to biggest, hence keep the last 10404 values which are the 10404 biggest variacnes

#Clear RAM space
del (full_d, var)

In [3]:
#Threshold train/test data and drop low variance values
train_d = pd.read_pickle('../DATA/train_data.pkl')
train_l = pd.read_pickle('../DATA/train_labels.pkl')

#Thresholding
train_d = np.log2(train_d.drop(ind2drop)+1)
train_genes = train_d.index

#Data normalization
stds = np.std(train_d, axis=1)
means = np.mean(train_d, axis=1)

# (x - x.mean)/x.std
train_d = (train_d.sub(means, axis=0)).div(stds, axis=0)

In [7]:
fp = np.memmap('train.dat', dtype='float64', mode='w+', shape=train_d.T.shape)
fp[:] = train_d.values[:].T
del fp

In [8]:
#Save each sample as a row, to be read off later by an iterator
t_start = time.time()
(train_d.T).to_csv('train.csv') #can be slow! ~205 seconds
print(time.time()-t_start)

del train_d

#test = pd.read_csv('train.csv', index_col = 0)

228.34806323051453


In [8]:
#Same for testset
test_d = pd.read_pickle('../DATA/test_data.pkl')
test_l = pd.read_pickle('../DATA/test_labels.pkl')

#Thresholding
test_d = np.log2(test_d.drop(ind2drop)+1)
test_genes = test_d.index

#Data normalization - using same mean and std from train set
# (x - x.mean)/x.std
test_d = (test_d.sub(means, axis=0)).div(stds, axis=0)

In [9]:
fp = np.memmap('test.dat', dtype='float64', mode='w+', shape=test_d.T.shape)
fp[:] = test_d.values[:].T
del fp

In [12]:
t_start = time.time()
(test_d.T).to_csv('test.csv') #can be slow!
print(time.time()-t_start)

del test_d

if any(~np.array(train_genes==test_genes)):
    print("Genes do not match")
    
#test = pd.read_csv('test.csv', index_col = 0)

57.473193883895874
